# Supply Chain Data Analysis

This notebook fetches data from both the Inventory Management System (IMS) and Order Management System (OMS) APIs, combines the data, and performs analysis.

In [ ]:
import pandas as pd
import numpy as np
import requests
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

## Fetch Data from APIs

In [ ]:
# Connect to IMS database
ims_conn = sqlite3.connect('ims.db')
inventory_data = pd.read_sql_query("""
    SELECT 
        p.id as product_id,
        p.name as product_name,
        p.category,
        p.price,
        p.quantity as current_stock,
        p.created_at,
        p.updated_at
    FROM products p
""", ims_conn)

# Connect to OMS database
oms_conn = sqlite3.connect('oms.db')
order_data = pd.read_sql_query("""
    SELECT 
        o.id as order_id,
        o.product_id,
        o.quantity as ordered_quantity,
        o.total_price,
        o.status,
        o.created_at as order_date
    FROM orders o
""", oms_conn)

## Data Processing and Analysis

In [ ]:
# Merge inventory and order data
merged_data = pd.merge(order_data, inventory_data, on='product_id')

# Convert timestamps
merged_data['order_date'] = pd.to_datetime(merged_data['order_date'])
merged_data['created_at'] = pd.to_datetime(merged_data['created_at'])
merged_data['updated_at'] = pd.to_datetime(merged_data['updated_at'])

# Calculate key metrics
merged_data['revenue'] = merged_data['ordered_quantity'] * merged_data['price']
merged_data['profit_margin'] = (merged_data['total_price'] - (merged_data['price'] * merged_data['ordered_quantity'])) / merged_data['total_price'] * 100

## Visualizations

In [ ]:
# Set plotting style
plt.style.use('seaborn')

# 1. Daily Order Volume
daily_orders = merged_data.groupby(merged_data['order_date'].dt.date).size()
plt.figure(figsize=(15, 6))
daily_orders.plot(kind='line')
plt.title('Daily Order Volume')
plt.xlabel('Date')
plt.ylabel('Number of Orders')
plt.show()

# 2. Product Category Distribution
plt.figure(figsize=(10, 6))
merged_data['category'].value_counts().plot(kind='bar')
plt.title('Orders by Product Category')
plt.xlabel('Category')
plt.ylabel('Number of Orders')
plt.xticks(rotation=45)
plt.show()

# 3. Stock vs Orders Analysis
stock_analysis = merged_data.groupby('product_name').agg({
    'current_stock': 'first',
    'ordered_quantity': 'sum'
}).reset_index()

plt.figure(figsize=(12, 6))
plt.scatter(stock_analysis['current_stock'], stock_analysis['ordered_quantity'])
plt.title('Current Stock vs Total Orders')
plt.xlabel('Current Stock')
plt.ylabel('Total Ordered Quantity')
plt.show()

## Inventory Analysis

In [ ]:
# Calculate inventory metrics
inventory_metrics = merged_data.groupby('product_name').agg({
    'current_stock': 'first',
    'ordered_quantity': ['sum', 'count'],
    'revenue': 'sum',
    'profit_margin': 'mean'
}).round(2)

inventory_metrics.columns = ['Current Stock', 'Total Orders', 'Order Count', 'Total Revenue', 'Avg Profit Margin %']
inventory_metrics['Stock Turnover Ratio'] = inventory_metrics['Total Orders'] / inventory_metrics['Current Stock']

# Display top 10 products by revenue
print("Top 10 Products by Revenue:")
print(inventory_metrics.sort_values('Total Revenue', ascending=False).head(10))

## Save Processed Data

In [ ]:
# Save the processed data to CSV
merged_data.to_csv('supply_chain_data.csv', index=False)
inventory_metrics.to_csv('inventory_metrics.csv')

print("Data analysis complete and saved to CSV files.")